In [27]:
import pandas as pd
import json
import numpy as np
#from collections import defaultdict
#from collections import Counter
import sklearn.linear_model
from sklearn.metrics import precision_recall_fscore_support as score
#import os
pd.options.mode.chained_assignment = None

In [2]:
### Purpose: get smaller-than-actual versions of each CSV, take a look at what we can get from each

sourceFilePath = "C:\\Users\\zange\\OneDrive\\Documents\\Business\\Education\\Drexel\\CS613\\dota-2-matches"

In [3]:
match_outcomes = pd.read_csv(open(sourceFilePath+"\\match_outcomes.csv"))

In [20]:
player_ratings = pd.read_csv(open(sourceFilePath+"\\player_ratings.csv"))

In [5]:
match_outcomes.head()

,match_id,account_id_0,account_id_1,account_id_2,account_id_3,account_id_4,start_time,parser_version,win,rad
0,1636204962,34549,0,0,-51743434,-120875154,1437014585,12,1,0
1,1636204962,0,61598,138825,0,207232,1437014585,12,0,1
2,1636322679,0,-44943233,-240360907,19599,0,1437019968,12,0,0
3,1636322679,-97530201,0,0,0,-116349387,1437019968,12,1,1
4,1637385965,0,0,0,104738,0,1437052551,12,1,0


In [6]:
player_ratings.head()

,account_id,total_wins,total_matches,trueskill_mu,trueskill_sigma
0,236579,14,24,27.868035,5.212361
1,-343,1,1,26.544163,8.065475
2,-1217,1,1,26.521103,8.114989
3,-1227,1,1,27.248025,8.092217
4,-1284,0,1,22.931016,8.092224


In [77]:
## The goal is to get each match on a single line with player accounts and their trueskill_mu and trueskill_sigma all together
## That has several parts
###1. break match_outcomes into two separate dataframes: one for Radiant, one for Dire
###2. Expand each separate dataframe with the trueskill markers for each player
###3. For each given match_id, re-stitch the two dataframes together into one frame.
########Be very careful to correctly identify which team is Radiant AND which team won.
radiant_mask = [rad==1 for rad in match_outcomes['rad']]
dire_mask = [rad==0 for rad in match_outcomes['rad']]

df_radiant = match_outcomes[radiant_mask]
df_dire = match_outcomes[dire_mask]

print("Match Outcomes: ",match_outcomes.shape)
print("Radiant: ",df_radiant.shape)
print("Dire: ",df_dire.shape)

Match Outcomes:  (1828588, 10)
Radiant:  (914294, 10)
Dire:  (914294, 10)


In [78]:
### to make data more manageable, slice radiant/dire dataframes to just 50k matches

df_radiant = df_radiant.head(50000)
df_dire = df_dire.head(50000)

In [79]:
print("Radiant: ",df_radiant.shape)
print("Dire: ",df_dire.shape)

Radiant:  (50000, 10)
Dire:  (50000, 10)


In [80]:
#######Actual runtime: 45 minutes
#######player_ratings[player_ratings['account_id']==0]
# This is a problem. account_id==0 is a number given to players who do not allow opendota to specifically identify their account during exports.
# For now, we will just roll with account_id==0 as if it were a person who has played 3315071 matches and take their trueskill_mu and trusekill_sigma
# as an average for the average player with account_id==0 and see where we go from there

df_radiant['ac0_trueskillmu'] = 0.0
df_radiant['ac0_trueskillsigma'] = 0.0
df_radiant['ac1_trueskillmu'] = 0.0
df_radiant['ac1_trueskillsigma'] = 0.0
df_radiant['ac2_trueskillmu'] = 0.0
df_radiant['ac2_trueskillsigma'] = 0.0
df_radiant['ac3_trueskillmu'] = 0.0
df_radiant['ac3_trueskillsigma'] = 0.0
df_radiant['ac4_trueskillmu'] = 0.0
df_radiant['ac4_trueskillsigma'] = 0.0

df_dire['ac0_trueskillmu'] = 0.0
df_dire['ac0_trueskillsigma'] = 0.0
df_dire['ac1_trueskillmu'] = 0.0
df_dire['ac1_trueskillsigma'] = 0.0
df_dire['ac2_trueskillmu'] = 0.0
df_dire['ac2_trueskillsigma'] = 0.0
df_dire['ac3_trueskillmu'] = 0.0
df_dire['ac3_trueskillsigma'] = 0.0
df_dire['ac4_trueskillmu'] = 0.0
df_dire['ac4_trueskillsigma'] = 0.0

for idx,row in df_radiant.iterrows():
    df_radiant.at[idx,'ac0_trueskillmu'] = float(player_ratings[player_ratings['account_id']==row['account_id_0']]['trueskill_mu'])
    df_radiant.at[idx,'ac1_trueskillmu'] = float(player_ratings[player_ratings['account_id']==row['account_id_1']]['trueskill_mu'])
    df_radiant.at[idx,'ac2_trueskillmu'] = float(player_ratings[player_ratings['account_id']==row['account_id_2']]['trueskill_mu'])
    df_radiant.at[idx,'ac3_trueskillmu'] = float(player_ratings[player_ratings['account_id']==row['account_id_3']]['trueskill_mu'])
    df_radiant.at[idx,'ac4_trueskillmu'] = float(player_ratings[player_ratings['account_id']==row['account_id_4']]['trueskill_mu'])
    df_radiant.at[idx,'ac0_trueskillsigma'] = float(player_ratings[player_ratings['account_id']==row['account_id_0']]['trueskill_sigma'])
    df_radiant.at[idx,'ac1_trueskillsigma'] = float(player_ratings[player_ratings['account_id']==row['account_id_1']]['trueskill_sigma'])
    df_radiant.at[idx,'ac2_trueskillsigma'] = float(player_ratings[player_ratings['account_id']==row['account_id_2']]['trueskill_sigma'])
    df_radiant.at[idx,'ac3_trueskillsigma'] = float(player_ratings[player_ratings['account_id']==row['account_id_3']]['trueskill_sigma'])
    df_radiant.at[idx,'ac4_trueskillsigma'] = float(player_ratings[player_ratings['account_id']==row['account_id_4']]['trueskill_sigma'])
    
for idx,row in df_dire.iterrows():
    df_dire.at[idx,'ac0_trueskillmu'] = float(player_ratings[player_ratings['account_id']==row['account_id_0']]['trueskill_mu'])
    df_dire.at[idx,'ac1_trueskillmu'] = float(player_ratings[player_ratings['account_id']==row['account_id_1']]['trueskill_mu'])
    df_dire.at[idx,'ac2_trueskillmu'] = float(player_ratings[player_ratings['account_id']==row['account_id_2']]['trueskill_mu'])
    df_dire.at[idx,'ac3_trueskillmu'] = float(player_ratings[player_ratings['account_id']==row['account_id_3']]['trueskill_mu'])
    df_dire.at[idx,'ac4_trueskillmu'] = float(player_ratings[player_ratings['account_id']==row['account_id_4']]['trueskill_mu'])
    df_dire.at[idx,'ac0_trueskillsigma'] = float(player_ratings[player_ratings['account_id']==row['account_id_0']]['trueskill_sigma'])
    df_dire.at[idx,'ac1_trueskillsigma'] = float(player_ratings[player_ratings['account_id']==row['account_id_1']]['trueskill_sigma'])
    df_dire.at[idx,'ac2_trueskillsigma'] = float(player_ratings[player_ratings['account_id']==row['account_id_2']]['trueskill_sigma'])
    df_dire.at[idx,'ac3_trueskillsigma'] = float(player_ratings[player_ratings['account_id']==row['account_id_3']]['trueskill_sigma'])
    df_dire.at[idx,'ac4_trueskillsigma'] = float(player_ratings[player_ratings['account_id']==row['account_id_4']]['trueskill_sigma'])

print("COMPLETE")

COMPLETE


In [81]:
## For backing up to save 45 minutes processing time
df_radiant.to_csv(open("df_radiant_d1.csv","w"))
df_dire.to_csv(open("df_dire_d1.csv","w"))

In [3]:
# Moved to Excel to more easily remove/rename columns,ended up with the following:
preprocessed_XandY = pd.read_csv(open("compiledX_noAccounts.csv"),index_col=0)

In [4]:
preprocessed_XandY.keys()

Index(['match_id', 'Dire_ac0_trueskillmu', 'Dire_ac0_trueskillsigma',
       'Dire_ac1_trueskillmu', 'Dire_ac1_trueskillsigma',
       'Dire_ac2_trueskillmu', 'Dire_ac2_trueskillsigma',
       'Dire_ac3_trueskillmu', 'Dire_ac3_trueskillsigma',
       'Dire_ac4_trueskillmu', 'Dire_ac4_trueskillsigma',
       'Dire_meanTrueSkillmu', 'Dire_meanTrueSkillsigma',
       'Radi_ac0_trueskillmu', 'Radi_ac0_trueskillsigma',
       'Radi_ac1_trueskillmu', 'Radi_ac1_trueskillsigma',
       'Radi_ac2_trueskillmu', 'Radi_ac2_trueskillsigma',
       'Radi_ac3_trueskillmu', 'Radi_ac3_trueskillsigma',
       'Radi_ac4_trueskillmu', 'Radi_ac4_trueskillsigma',
       'Radi_meanTrueSkillmu', 'Radi_meanTrueSkillsigma', 'Radi_win'],
      dtype='object')

In [5]:
###Now that we have restitched everything together, it's time to figure out how best to perform this logistic regression
### Let's pull the four features we want into a numpy array
X = pd.DataFrame(preprocessed_XandY['Dire_meanTrueSkillmu'])
X['Dire_meanTrueSkillsigma'] = preprocessed_XandY['Dire_meanTrueSkillsigma']
X['Radi_meanTrueSkillmu'] = preprocessed_XandY['Radi_meanTrueSkillmu']
X['Radi_meanTrueSkillsigma'] = preprocessed_XandY['Radi_meanTrueSkillsigma']

In [6]:
X = X.values

In [7]:
Y = pd.DataFrame(preprocessed_XandY['Radi_win'])
Y = Y.values

In [8]:
# So now that we have these X and Y ready to go, we can get started with the total process.
X

array([[25.21466116,  6.5180607 , 24.99506614,  6.12051626],
       [25.25767538,  6.67969932, 26.71221832,  8.06249729],
       [25.72116133,  7.53599049, 24.59110219,  7.95222144],
       ...,
       [30.6747832 ,  6.34965472, 23.90165202,  7.85640872],
       [25.80935994,  6.03820578, 26.32776747,  6.33685745],
       [23.57007334,  4.39916558, 25.26516682,  6.42260842]])

In [9]:
# 2/3 training, 1/3 testing
trainingLen = int(np.ceil(X.shape[0]*(2/3)))
Xtrain = X[0:trainingLen,:]
Xtest = X[trainingLen:,:]
Ytrain = Y[0:trainingLen,:]
Ytest = Y[trainingLen:,:]

In [10]:
XtrainMeans = np.mean(Xtrain,axis=0)
XtrainStds = np.std(Xtrain,axis=0)

In [11]:
XtrainNorm = (Xtrain - XtrainMeans)*1./XtrainStds ## normalize
XtrainNorm

array([[-0.09090512, -0.00404834, -0.25794848, -0.35804378],
       [-0.06186882,  0.12206439,  0.89897319,  1.16619501],
       [ 0.25100254,  0.79015481, -0.53011693,  1.07964075],
       ...,
       [-0.46908378,  1.31506998,  1.64773502,  0.21535685],
       [ 1.3384113 , -1.63457597,  1.12026356, -0.76113133],
       [-0.41248459,  0.19976711, -0.82027765,  0.92366979]])

In [12]:
XtestNorm = (Xtest - XtrainMeans)*1./XtrainStds ## normalize
XtestNorm

array([[-1.39131924,  0.34787375,  0.95183378, -1.32514371],
       [ 1.24738136,  0.85845012, -0.25462432,  1.37877114],
       [-0.34258088,  1.27299944, -0.90105093, -0.15334658],
       ...,
       [ 3.5948934 , -0.13544105, -0.99463012,  1.00443844],
       [ 0.3105401 , -0.3784379 ,  0.63995158, -0.18824004],
       [-1.201067  , -1.65724013, -0.07596966, -0.12093508]])

In [13]:
logreg = sklearn.linear_model.LogisticRegression()

In [14]:
logreg.fit(Xtrain,Ytrain)

C:\Users\zange\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\zange\Anaconda3\lib\site-packages\sklearn\utils\validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='warn',
          tol=0.0001, verbose=0, warm_start=False)

In [15]:
logreg

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='warn',
          tol=0.0001, verbose=0, warm_start=False)

In [16]:
sc = logreg.score(Xtrain,Ytrain)

In [17]:
p = logreg.predict(Xtest)

In [18]:
p

array([1, 0, 0, ..., 0, 1, 1], dtype=int64)

In [19]:
Ytest

array([[1],
       [0],
       [0],
       ...,
       [0],
       [0],
       [1]], dtype=int64)

In [20]:
results = pd.DataFrame(Ytest)

In [21]:
results.rename(columns={0:'Ytest'},inplace=True)

In [22]:
results['Output'] = p.T

In [23]:
results.head()

,Ytest,Output
0,1,1
1,0,0
2,0,0
3,0,0
4,1,0


In [28]:
skprecision, skrecall, skfscore, sksupport = score(results['Ytest'],results['Output'])

In [24]:
TP = 0
TN = 0
FP = 0
FN = 0

for idx,row in results.iterrows():
    if row['Ytest']==1:
        if row['Ytest']==row['Output']:
            TP = TP + 1
        else:
            FP = FP + 1
    else:
        if row['Ytest']==row['Output']:
            TN = TN + 1
        else:
            FN = FN + 1

In [25]:
precision = TP/(float(TP)+FP)
recall = TP/(float(TP)+FN)
fMeasure = (2*precision*recall)/(precision+recall)
accuracy = (TP+TN)/(float(TP)+TN+FP+FN)
print("Precision: ",precision)
print("Recall: ",recall)
print("fMeasure: ",fMeasure)
print("Accuracy: ",accuracy)

Precision:  0.7783865702961064
Recall:  0.7583191368540602
fMeasure:  0.7682218259218777
Accuracy:  0.7582503300132005


In [26]:
sc

0.7468350632987341

In [29]:
print("skPrecision: ",skprecision)
print("skRecall: ",skrecall)
print("skfMeasure: ",skfscore)
print("skSupport: ",sksupport)

skPrecision:  [0.75817326 0.75831914]
skRecall:  [0.73689416 0.77838657]
skfMeasure:  [0.74738228 0.76822183]
skSupport:  [8088 8578]


In [41]:
testingRuns = []

In [50]:
### running the regression with multiple testing samples
allTols = [.0001]
for inputTol in allTols:
    logreg = sklearn.linear_model.LogisticRegression(penalty='l2',dual=False,tol=inputTol, \
                                                     C=1.0,fit_intercept=True,intercept_scaling=1, \
                                                     class_weight='balanced',random_state=0, \
                                                     solver='liblinear',max_iter=100,multi_class='ovr', \
                                                     verbose=0,warm_start=False,n_jobs=None)
    logreg.fit(Xtrain,np.ravel(Ytrain))
    sc = logreg.score(Xtrain,np.ravel(Ytrain))
    p = logreg.predict(Xtest)
    results = pd.DataFrame(Ytest)
    results.rename(columns={0:'Ytest'},inplace=True)
    results['Output'] = p.T
    TP = 0
    TN = 0
    FP = 0
    FN = 0
    
    for idx,row in results.iterrows():
        if row['Ytest']==1:
            if row['Ytest']==row['Output']:
                TP = TP + 1
            else:
                FP = FP + 1
        else:
            if row['Ytest']==row['Output']:
                TN = TN + 1
            else:
                FN = FN + 1
    
    precision = TP/(float(TP)+FP)
    recall = TP/(float(TP)+FN)
    fMeasure = (2*precision*recall)/(precision+recall)
    accuracy = (TP+TN)/(float(TP)+TN+FP+FN)
    print("Coefficient: ",logreg.coef_)
    print("Intercept: ", logreg.intercept_)
    print("Classes: ",logreg.classes_)
    print("Precision: ",precision)
    print("Recall: ",recall)
    print("fMeasure: ",fMeasure)
    print("Accuracy: ",accuracy)
    testingRuns.append((logreg,(precision,recall,fMeasure,accuracy)))

Coefficient:  [[-0.70434053 -0.02324042  0.73551793  0.03208093]]
Intercept:  [-0.83755776]
Classes:  [0 1]
Precision:  0.7583352762881791
Recall:  0.7698224852071006
fMeasure:  0.7640357058961711
Accuracy:  0.7589103564142565
